## `animation_hadgem-ll_historic.ipynb`: create animation for historic hadgem-ll model simulation

In [1]:
import context
import warnings
import intake
import xarray as xr 
import matplotlib.pyplot as plt 
import pandas as pd
import cftime
import gcsfs
import cartopy.crs as ccrs
from pathlib import Path
import pandas as pd
from a448_lib import data_read
import fsspec
import cmocean as cm
import cartopy.feature as cfeature
import numpy as np
import warnings
import matplotlib.animation as animation
import matplotlib.path as mpath

SyntaxError: invalid syntax (<fstring>, line 1)

### Grab json file with all of the data from cmip6

* Download the catalog in csv and json format

In [ ]:
csv_filename = "pangeo-cmip6.csv"
root = "https://storage.googleapis.com/cmip6"
if Path(csv_filename).is_file():
    print(f"found {csv_filename}")
else:
    print(f"downloading {csv_filename}")
    data_read.download(csv_filename,root=root)
    
json_filename="https://storage.googleapis.com/cmip6/pangeo-cmip6.json"

* make a dataframe from the csv version

In [ ]:
catalog_df=pd.read_csv(csv_filename)
catalog_df.head()

* make an intake collection from the json version

In [ ]:
col = intake.open_esm_datastore(json_filename)

In [ ]:
col

## First show all MOHC historical runs

In [ ]:
source = "HadGEM3-GC31-LL"
query = dict(
    experiment_id=['historical'],
    institution_id = "MOHC",
    source_id = source,
    table_id=["SImon"],
    variable_id=['sithick'])

col_subset = col.search(require_all_on=["source_id"],**query)

In [ ]:
col_subset.df.head()

In [ ]:
len(col_subset.df)

list_of_members = col_subset.df

## get the first realization for the sithick dataset

In [ ]:
member = 'r1i1p1f3'
filename=col_subset.df.query("member_id=='r1i1p1f3'")['zstore'].iloc[0]

In [ ]:
dset_mohc_sithick=xr.open_zarr(fsspec.get_mapper(filename), consolidated=True)
dset_mohc_sithick

## Now get the cell area for the ocean grid

In [ ]:
query = dict(
    experiment_id=['piControl'],
    institution_id = "MOHC",
    table_id = "Ofx",
    source_id = source,
    member_id = 'r1i1p1f1',
    variable_id=['areacello'])

col_subset = col.search(require_all_on=["source_id"],**query)
col_subset.df

In [ ]:
filename=col_subset.df['zstore'].iloc[0]
filename

In [ ]:
dset_mohc_areacello=xr.open_zarr(fsspec.get_mapper(filename), consolidated=True)
dset_mohc_areacello

## Plot the lat/lon for this curvilinear ocean grid

In [ ]:
lons = dset_mohc_sithick.longitude
lats = dset_mohc_sithick.latitude
data = dset_mohc_sithick['sithick']
animation_data = data

In [ ]:
lons.shape
lats.shape
data.shape

In [ ]:
plt.plot(lons[-30:],lats[-30:],'r.');

In [ ]:
def deseam(lon, lat, data):
    """
    Function to get rid of the "seam" that shows up on 
    the map when you're using these curvilinear grids.
    """
    i, j = lat.shape
    new_lon = np.zeros((i, j + 1))
    new_lon[:, :-1] = lon
    new_lon[:, -1] = lon[:, 0]

    new_lat = np.zeros((i, j + 1))
    new_lat[:, :-1] = lat
    new_lat[:, -1] = lat[:, 0]

    new_data = np.zeros((i, j + 1))
    new_data[:, :-1] = data
    new_data[:, -1] = data[:, 0]
    new_data = np.ma.array(new_data, mask=np.isnan(new_data))
    return new_lon, new_lat, new_data

In [ ]:
lons, lats, newdata = deseam(lons,lats,data[0,:,:])

### Create Animation

In [ ]:
# time step
time_step = 120
years = [0,1*time_step, 2*time_step, 3*time_step, 4*time_step, 5*time_step, 6*time_step, 7*time_step, 8*time_step, 9*time_step, 10*time_step, 11*time_step, 12*time_step, 13*time_step,14*time_step, 15*time_step, 16*time_step]
dset_mohc_sithick['sithick'][years,:,:]

every_ten_years = dset_mohc_sithick['sithick'][years,:,:]
every_ten_years_time = dset_mohc_sithick['time'][years]

In [ ]:
%matplotlib notebook
import matplotlib.animation as animation

f, ax = plt.subplots(1,1,figsize=(6,6),
                     subplot_kw=dict(projection=ccrs.Orthographic(0, 80)))

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

ax.set_boundary(circle, transform=ax.transAxes)

ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())
cax = ax.pcolormesh(lons,
              lats,
              dset_mohc_sithick['sithick'][0,:,:],
              transform=ccrs.PlateCarree(),
              vmin=0, vmax=12, cmap='ocean_r')

f.colorbar(cax, label='sea ice thickness (m)')

t = animation_data.time


#ax.set_title('CCCma sea ice thickness (m)' + str(dset_cccma_sithick['time'][i]))

# Add land.
ax.add_feature(cfeature.LAND, color='#444444', zorder=4);
ax.gridlines(color = "white", zorder=5)

def animate(i):
        cax.set_array(every_ten_years[i, :, :].values.flatten())
        ax.set_title('Sea ice thickness (m) :' + str(every_ten_years_time[i].values))
        

        
        
anim = animation.FuncAnimation(f, animate, interval=50, frames=len(t)-1, repeat=False, blit=True)

f.show()
animation_type = 'pcolor'
anim.save('hadgem-LL_historic.gif')